<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/examples_for_ppl_who_asked/drainage_area_on_arran.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Drainage area on Arran: colab version

Prepared by Simon M Mudd, 20-Sept-2024

## Get lsdtopotools and lsdviztools

This installs lsdtopotools and lsdviztools in google colab.
`lsdtopotools` command line is installed from the university of edinburgh's debian server.

`lsdviztools` is installed via `pip`

This will take around 25 seconds.

In [1]:
!wget https://pkgs.geos.ed.ac.uk/geos-jammy/pool/world/l/lsdtopotools2/lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null
!apt install ./lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null

In [2]:
!pip install lsdviztools &> /dev/null

Check the versions.
This notebook was written with lsdviztools 0.4.12
and lsdtopotools version 0.9

In [ ]:
import lsdviztools
lsdviztools.__version__

In [ ]:
!lsdtt-basic-metrics -v

## Get gdal command line tools

We might need some gdal command line tools. This is used for raster manipulation. Installation will take maybe 10 seconds.

In [ ]:
!apt install gdal-bin &> /dev/null

!gdalinfo --version

## Mount your DEM from google drive

You will need to mount your DEM from google drive. Note that uploading the DEM directly into google colab (i.e., by dragging and dropping the file into the file system in a colab window) will take *forever*. So the recommended approach is to upload the DEM and driver file into a file folder in google drive and then mount it.

*Note: you will need to give google colab permission to do this on your drive.*

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

### Navigate to the directory where you keep the data.

You will need to modify the path below for your own data.
Note that the first three subdirectories on a google drive will always be `/content/drive/MyDrive/`

In [ ]:
%cd /content/drive/MyDrive/dissertation/

Lets just make sure this works by checking the working directory and the files:

You should see the DEM and your driver file.

In [ ]:
!pwd
!ls

### Deal with .tif

If you start with a .tif file, you need to convert it to the correct coordinate system and also convert it to an ENVI bil file format.

I use the UTM zone for Arran here, which is 30 (although right on the edge with 29).

You will need to check your DEM grid resolution and alter it below in the `-tr` part. So if the grid spacing is 10m by 10m you should use `-tr 10 10`.

In [ ]:
!gdalwarp -t_srs '+proj=utm +zone=30 +north +datum=WGS84' -of ENVI -r cubic -tr 10 10 arran.tif arran.bil

## Get the drainage area

Now we will get the drainage area. We need a driver file. We can make one with the following script.

In [5]:
import lsdviztools.lsdmapwrappers as lsdmw

lsdtt_parameters = {"print_d8_drainage_area_raster" : "true",
                    "write_hillshade" : "true",
                    "remove_seas": "true"}
r_prefix = "arran"
w_prefix = "arran"
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.write_lsdtt_driver()

!ls *.driver

The lsdtopotools command line tools available are: 
['lsdtt-basic-metrics', 'lsdtt-channel-extraction', 'lsdtt-chi-mapping', 'lsdtt-cosmo-tool', 'lsdtt-hillslope-channel-coupling', 'lsdtt-valley-metrics']
Testing has been done against lsdtopotools v0.7
Done writing the driver file
Test_01.driver


In [8]:
!cat Test_01.driver

# This is an LSDTopoTools driver file written with the python package lsdviztools

# File locations
read path: ./
write path: ./
read fname: arran
write fname: arran


# Now for the parameters
print_dinf_drainage_area_raster: true
write_hillshade: true


Now run the analysis:

In [ ]:
!lsdtt-basic-metrics Test_01.driver

You should now have a file with the drainage area!